# Integer Levinson Algoritm

Implementation of "LEVINSON ALGORITHM OVER INTEGERS FOR STRONGLY REGULAR HERMITIAN TOEPLITZ MATRICES" by Segalov and Bistritz.

The [Levinson Algorithm](https://en.wikipedia.org/wiki/Levinson_recursion) is a procedure in linear algebra to recursively calculate the solution to an equation involving a Toeplitz matrix. The matrix structure is taken advantage of to get a better complexity than classic Gauss reduction.

The following version works purely in integers.

$$
R_n = \begin{pmatrix}
r_0 & r_1 & ... & r_n\\
r_1* & r_0 & ... & r_{n-1}\\
\vdots &  & \ddots & \\
r_n* & r_{n-1}* & ... & r_0\\
\end{pmatrix}

\\
\\

R_n \cdot \begin{bmatrix}
\alpha_0 \\
\vdots \\
\alpha_{n-1} \\
1
\end{bmatrix}

=

\begin{bmatrix}
0 \\
\vdots \\
0 \\
E_n
\end{bmatrix}



$$

Define polynomial class:

In [43]:
class Polynomial:
    def __init__(self, coeffs = [1]): #init to 1
            self.coeffs = coeffs
            
    def __rmul__(self, other): #multiplication by scalar
        new_coeffs = []
        for c in self.coeffs:
            new_coeffs += [c * other]
        return Polynomial(new_coeffs)
    
    def __truediv__(self, other): #divison by scalar
        new_coeffs = []
        for c in self.coeffs:
            new_coeffs += [c // other]
        return Polynomial(new_coeffs)

    
    def __add__(self, other): # add to other polynom
        cs1 = self.coeffs
        cs2 = other.coeffs
        if len(cs2)<len(cs1):
            cs2 = cs2 + [0]*(len(cs1)-len(cs2))
        elif len(cs1)<len(cs2):
            cs1 = cs1 + [0]*(len(cs2)-len(cs1))
        res = [sum(x) for x in zip(cs1, cs2)]
        return Polynomial(res)
    
    def __sub__(self, other):
        return self + (-1)*other
    
    def multiplybyz(self):
        return Polynomial([0] + self.coeffs)
    
    def reciprocal(self):
        new_coeffs = [x.conjugate() for x in self.coeffs[::-1]]
        return Polynomial(new_coeffs)

    def __call__(self, z):
        res = 0
        for coeff in self.coeffs[::-1]:
            res = res * z + coeff
        return res 
    
    def levinson_eval(self, rs):
        res = 0
        for i, c in enumerate(self.coeffs):
            res += c * r[i+1]
        return res
    
    def __str__(self):
        s = ''
        for p, coeff in enumerate(self.coeffs[::-1]):
            p1 = len(self.coeffs) - 1 - p
            if coeff!=0:
                if coeff==1 and p1>0:
                    s_tmp = ''
                else:
                    s_tmp = str(coeff)
                if p1>0:
                    s_tmp += 'z'
                if p1>1:
                    s_tmp +='^'+str(p1)               
                    
                if s=='':
                    s = s_tmp
                else:
                    s += ' + ' + s_tmp
        return s
    
    def __repr__(self):
        return self.__str__()

In [23]:
a = Polynomial([1,2,3])
print(a)
print(a.multiplybyz())
print(5*a)

3z^2 + 2z + 1
3z^3 + 2z^2 + z
15z^2 + 10z + 5


In [60]:
r = list(range(1,30+1)[::-1])
#from scipy.linalg import toeplitz
#print(toeplitz(r))

f = Polynomial([1])
e = r[0]
d = r[1]
e_prev = 1
print(f'0 | f={f}  e={e} d={d}  ')
for m in range(1, len(r)):
    f = (e * f.multiplybyz() - d * f.reciprocal()) / f.coeffs[-1]
    e, e_prev = (e**2 - abs(d)**2) // e_prev , e
    if m<(len(r)-1):
        d = f.levinson_eval(r)
    else:
        d = None
    print(f'{m} | f={f}  e={e} d={d}  ')

print("Solution:")
from fractions import Fraction
alphas_str = [str(Fraction(x,f.coeffs[-1]))  for x in f.coeffs]
En = Fraction(e, f.coeffs[-1])
print(f'alpha = {alphas_str}, En={En}')


0 | f=1  e=30 d=29  
1 | f=30z + -29  e=59 d=-1  
2 | f=59z^2 + -58z + 1  e=116 d=-2  
3 | f=116z^3 + -114z^2 + 2  e=228 d=-4  
4 | f=228z^4 + -224z^3 + 4  e=448 d=-8  
5 | f=448z^5 + -440z^4 + 8  e=880 d=-16  
6 | f=880z^6 + -864z^5 + 16  e=1728 d=-32  
7 | f=1728z^7 + -1696z^6 + 32  e=3392 d=-64  
8 | f=3392z^8 + -3328z^7 + 64  e=6656 d=-128  
9 | f=6656z^9 + -6528z^8 + 128  e=13056 d=-256  
10 | f=13056z^10 + -12800z^9 + 256  e=25600 d=-512  
11 | f=25600z^11 + -25088z^10 + 512  e=50176 d=-1024  
12 | f=50176z^12 + -49152z^11 + 1024  e=98304 d=-2048  
13 | f=98304z^13 + -96256z^12 + 2048  e=192512 d=-4096  
14 | f=192512z^14 + -188416z^13 + 4096  e=376832 d=-8192  
15 | f=376832z^15 + -368640z^14 + 8192  e=737280 d=-16384  
16 | f=737280z^16 + -720896z^15 + 16384  e=1441792 d=-32768  
17 | f=1441792z^17 + -1409024z^16 + 32768  e=2818048 d=-65536  
18 | f=2818048z^18 + -2752512z^17 + 65536  e=5505024 d=-131072  
19 | f=5505024z^19 + -5373952z^18 + 131072  e=10747904 d=-262144  
20 | 

Verification of solution:

In [63]:
import numpy as np
from scipy.linalg import toeplitz
mat = toeplitz(r)
alphas = np.array([x/f.coeffs[-1]  for x in f.coeffs])
print(np.dot(mat,alphas))

[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     1.9375]
